## Install Dependencies

In [23]:
!apt-get update
!apt-get install -y ffmpeg
!pip install -U openai-whisper jiwer
!pip install -U tqdm

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

### Verify Installation

In [24]:
import whisper
import torch
import subprocess

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Whisper loaded:", whisper.load_model("base") is not None)

subprocess.run(["ffmpeg", "-version"], capture_output=True)
print("FFmpeg OK")

Torch: 2.9.0+cu126
CUDA available: True
Whisper loaded: True
FFmpeg OK


## Mount Google Drive

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import and Configuration

In [26]:

import json
import subprocess
from pathlib import Path
from typing import List, Dict, Tuple
from dataclasses import dataclass, asdict
import whisper
from jiwer import wer as compute_wer


CONFIG = {
    "cha_dir": Path("/content/drive/MyDrive/asr/data/cha/1"),
    "audio_dir": Path("/content/drive/MyDrive/asr/data/songs/1"),
    "output_dir": Path("/content/drive/MyDrive/asr/output/whisper_children_dataset"),
    "whisper_model": "base",
    "sample_size": None,  #
    "train_ratio": 0.8,
    "sample_rate": 16000,
    "audio_extensions": [".wav", ".mp3", ".m4a", ".flac"]
}

# CHILDES roles
CHILD_ROLES = {"Target_Child", "Child", "Sibling", "Peer", "Playmate"}
ADULT_ROLES = {"Investigator", "Teacher", "Mother", "Father", "Adult", "Caregiver", "Parent"}



## Zone 1: File Matching


COMPLETE PIPELINE NOTEBOOK
Input: Dataset folders (data/ca + data/songs)
Output: Training dataset for Whisper fine-tuning (children voices only)

Flow:
1. Match .cha ↔ Audio files
2. Extract .cha segments (word-level timestamps)
3. Segment audio files based on timestamps
4. Evaluate Whisper baseline (children only)
5. Calculate WER (children only)
6. Create training dataset (JSONL + metadata)


In [27]:
import re
from pathlib import Path
from dataclasses import dataclass
from typing import List, Union

@dataclass
class WorSegment:
    speaker: str
    text: str
    words: list  # [(word, start, end)]
    file_name: str = ""  # Ajouter le nom du fichier source


def extract_wor_segments(path: Union[Path, str], debug: bool = False) -> List[WorSegment]:
    """
    Extraire segments %wor d'un fichier .cha

    Args:
        path: Chemin vers un fichier .cha OU un dossier contenant des .cha
        debug: Afficher les infos de parsing

    Returns:
        Liste de WorSegment
    """
    path = Path(path)

    if path.is_dir():
        # Si c'est un dossier, traiter tous les .cha
        return _extract_from_directory(path, debug=debug)
    elif path.is_file():
        # Si c'est un fichier, le traiter
        return _extract_from_file(path, debug=debug)
    else:
        raise FileNotFoundError(f"Chemin invalide: {path}")


def _extract_from_directory(cha_dir: Path, debug: bool = False) -> List[WorSegment]:
    """Extraire de tous les fichiers .cha d'un dossier (récursivement)"""

    # Chercher les .cha dans le dossier ET les sous-dossiers
    cha_files = sorted(cha_dir.glob("*.cha")) + sorted(cha_dir.glob("**/*.cha"))
    # Enlever les doublons
    cha_files = sorted(set(cha_files))

    if not cha_files:
        print(f"Aucun fichier .cha trouvé dans {cha_dir}")
        return []

    if debug:
        print(f"Traitement de {len(cha_files)} fichiers .cha\n")

    all_segments = []

    for cha_file in cha_files:
        if debug:
            print(f" {cha_file.name}...", end=" ")

        segments = _extract_from_file(cha_file, debug=False)
        all_segments.extend(segments)

        if debug:
            print(f"({len(segments)} segments)")

    if debug:
        print(f"\n{'=' * 60}")
        print(f"Total: {len(all_segments)} segments de {len(cha_files)} fichiers")
        print(f"{'=' * 60}\n")

    return all_segments


In [28]:

def _extract_from_file(cha_file: Path, debug: bool = False) -> List[WorSegment]:
    """Extraire de un seul fichier .cha"""

    segments = []
    current_speaker = None
    file_name = cha_file.stem

    with cha_file.open(encoding="utf-8") as f:
        for line in f:
            line = line.rstrip()

            # ── tour principal
            if line.startswith("*"):
                current_speaker = line.split(":", 1)[0].replace("*", "").strip()

            # ── word tier
            elif line.startswith("%wor:"):
                if not current_speaker:
                    continue

                # Extraire la partie après "%wor:"
                wor_content = line.split(":", 1)[1].strip()

                # Nettoyer les caractères de contrôle
                wor_content = wor_content.replace('\x15', '')

                # Parser simple : splitter par espaces et apparier word + timestamp
                tokens = wor_content.split()

                words = []
                i = 0
                while i < len(tokens):
                    token = tokens[i]

                    # Vérifier si c'est un timestamp (format XXXXX_XXXXX)
                    if re.match(r"^\d{5,}_\d{5,}$", token):
                        # C'est un timestamp → l'attacher au mot précédent
                        if words:
                            word, _, _ = words[-1]
                            match = re.match(r"(\d+)_(\d+)", token)
                            if match:
                                start, end = int(match.group(1)), int(match.group(2))
                                words[-1] = (word, start, end)
                        i += 1
                        continue

                    # Sinon, c'est un mot
                    words.append((token, None, None))
                    i += 1

                # Filtrer : garder seulement les mots avec timestamps
                words_with_ts = [(w, s, e) for w, s, e in words if s is not None and e is not None]

                if not words_with_ts:
                    continue

                # Nettoyer le texte : enlever les ponctuations isolées
                clean_words = [w for w, _, _ in words_with_ts if w not in ('?', '.', ',', '!', '+...')]

                if clean_words:
                    clean_text = " ".join(clean_words)

                    segments.append(
                        WorSegment(
                            speaker=current_speaker,
                            text=clean_text,
                            words=words_with_ts,
                            file_name=file_name
                        )
                    )

                    if debug and len(segments) <= 3:
                        print(f"\n {current_speaker}")
                        print(f"   Text: {clean_text[:70]}")
                        print(f"   Words: {words_with_ts[:3]}...")

    if debug:
        print(f"\n{'=' * 60}")
        print(f"Total segments ({file_name}): {len(segments)}")
        print(f"{'=' * 60}")

    return segments

In [29]:
def print_statistics(segments: List[WorSegment]):
    """Afficher statistiques détaillées sur les segments"""

    if not segments:
        print(" Aucun segment trouvé")
        return

    print(f"\n{'=' * 60}")
    print("STATISTIQUES")
    print(f"{'=' * 60}")

    # Par speaker
    by_speaker = {}
    by_file = {}

    for seg in segments:
        # Par speaker
        by_speaker.setdefault(seg.speaker, []).append(seg)

        # Par fichier
        by_file.setdefault(seg.file_name, []).append(seg)

    print(f"\nPar speaker ({len(by_speaker)} speakers):")
    for speaker in sorted(by_speaker.keys()):
        segs = by_speaker[speaker]
        total_duration = sum(s.words[-1][2] - s.words[0][1] for s in segs) / 1000
        print(f"   {speaker:20} {len(segs):3d} segments | {total_duration:6.1f}s audio")

    print(f"\nPar fichier ({len(by_file)} fichiers):")
    for file_name in sorted(by_file.keys()):
        segs = by_file[file_name]
        total_duration = sum(s.words[-1][2] - s.words[0][1] for s in segs) / 1000
        print(f"   {file_name:30} {len(segs):3d} segments | {total_duration:6.1f}s audio")

    # Stats globales
    total_duration = sum(s.words[-1][2] - s.words[0][1] for s in segments) / 1000 / 60
    avg_words = sum(len(s.words) for s in segments) / len(segments)

    print(f"\nGlobales:")
    print(f"   Total segments: {len(segments)}")
    print(f"   Total audio: {total_duration:.1f} minutes")
    print(f"   Mots par segment (moyennes): {avg_words:.1f}")
    print(f"{'=' * 60}\n")


if __name__ == "__main__":
    # Exemple 1: Traiter UN fichier
    print("=" * 60)
    print("EXEMPLE 1: UN FICHIER")
    print("=" * 60)
    segments_single = extract_wor_segments(Path("/content/drive/MyDrive/asr/data/cha/1/01-1a.cha"), debug=True)

    # Exemple 2: Traiter UN DOSSIER ENTIER
    print("\n" + "=" * 60)
    print("EXEMPLE 2: DOSSIER ENTIER")
    print("=" * 60)
    segments_all = extract_wor_segments(Path("/content/drive/MyDrive/asr/data/cha/1/"), debug=True)

    # Afficher statistiques
    print_statistics(segments_all)

    # Exemples
    if segments_all:
        print(f"{'=' * 60}")
        print("EXEMPLES DE SEGMENTS:")
        print(f"{'=' * 60}")
        for i, seg in enumerate(segments_all[:5]):
            print(f"\n[{i + 1}] {seg.speaker} ({seg.file_name})")
            print(f"  Text: {seg.text}")
            if seg.words:
                print(f"  Time: {seg.words[0][1]} → {seg.words[-1][2]} ms")
                print(f"  Words: {seg.words[:3]}...")





EXEMPLE 1: UN FICHIER

 KAT
   Text: un escargot Dylan
   Words: [('un', 10438, 10478), ('escargot', 10739, 10919), ('Dylan', 10919, 11419)]...

 KAT
   Text: comment
   Words: [('comment', 35270, 35770)]...

 WIL
   Text: moi fais la fourmi moi
   Words: [('moi', 38747, 39670), ('fais', 40011, 40612), ('la', 41896, 42136)]...

Total segments (01-1a): 26

EXEMPLE 2: DOSSIER ENTIER
Traitement de 112 fichiers .cha

 01-1a.cha... (26 segments)
 01-1b.cha... (8 segments)
 01-2.cha... (211 segments)
 01-3a.cha... (91 segments)
 01-3b.cha... (71 segments)
 01-3c.cha... (89 segments)
 01-3d.cha... (181 segments)
 01-4.cha... (82 segments)
 02-5.cha... (217 segments)
 02-7.cha... (397 segments)
 02-8.cha... (8 segments)
 02-9a.cha... (224 segments)
 02-9b.cha... (118 segments)
 02-9c.cha... (40 segments)
 02-9d.cha... (138 segments)
 02-9e.cha... (17 segments)
 02-9f.cha... (67 segments)
 03-10a.cha... (75 segments)
 03-10b.cha... (130 segments)
 03-13a.cha... (66 segments)
 03-13b.cha... (98 

In [30]:
def find_matching_files(cha_dir: Path, audio_dir: Path, extensions: List[str]) -> Dict:
    """Match .cha with audio files by relative path (respects subdirectories)"""

    cha_files = sorted(cha_dir.glob("**/*.cha"))
    audio_files = []
    for ext in extensions:
        audio_files.extend(audio_dir.glob(f"**/*{ext}"))

    # Créer dicts: relative_path_with_stem → fichier
    # Exemple: "1/01-1a" pour data/cha/1/01-1a.cha
    cha_by_path = {}
    for f in cha_files:
        relative_stem = str(f.relative_to(cha_dir).with_suffix(""))  # "1/01-1a"
        cha_by_path[relative_stem] = f

    audio_by_path = {}
    for f in audio_files:
        relative_stem = str(f.relative_to(audio_dir).with_suffix(""))
        audio_by_path[relative_stem] = f

    # Matcher: chercher les mêmes chemins relatifs
    matched = []
    cha_missing = []
    audio_orphans = []

    for relative_path in cha_by_path:
        if relative_path in audio_by_path:
            matched.append((cha_by_path[relative_path], audio_by_path[relative_path]))
        else:
            cha_missing.append(cha_by_path[relative_path])

    for relative_path in audio_by_path:
        if relative_path not in cha_by_path:
            audio_orphans.append(audio_by_path[relative_path])

    return {
        "matched": matched,
        "cha_missing_audio": cha_missing,
        "audio_orphans": audio_orphans,
        "total_cha": len(cha_files),
        "total_audio": len(audio_files),
        "matched_count": len(matched)
    }


def print_matching_report(result: Dict):
    """Afficher le rapport de matching"""
    print("\n" + "="*70)
    print("STEP 1: FILE MATCHING (by relative path)")
    print("="*70)
    print(f"\n Found:")
    print(f"   Total .cha files:       {result['total_cha']}")
    print(f"   Total audio files:      {result['total_audio']}")
    print(f"   Matched pairs:        {result['matched_count']}")
    print(f"   .cha missing audio:  {len(result['cha_missing_audio'])}")
    print(f"   Audio orphans:       {len(result['audio_orphans'])}")

    if result['cha_missing_audio']:
        print(f"\n   Missing audio for:")
        for cha in result['cha_missing_audio'][:10]:
            print(f"      - {cha.name}")
        if len(result['cha_missing_audio']) > 10:
            print(f"      ... and {len(result['cha_missing_audio']) - 10} more")

    if result['audio_orphans']:
        print(f"\n   Audio without .cha:")
        for audio in result['audio_orphans'][:10]:
            print(f"      - {audio.name}")
        if len(result['audio_orphans']) > 10:
            print(f"      ... and {len(result['audio_orphans']) - 10} more")

    print("\n" + "="*70 + "\n")


## Zone 2: Segment Extraction

In [31]:
def extract_segments_from_matched(matched_pairs: List[Tuple[Path, Path]]) -> List[WorSegment]:
    """Extract .cha segments from matched files only"""

    print("="*70)
    print("STEP 2: EXTRACT .CHA SEGMENTS")
    print("="*70)

    all_segments = []

    for i, (cha_file, audio_file) in enumerate(matched_pairs):
        segments = extract_wor_segments(cha_file, debug=False)
        all_segments.extend(segments)

        if (i + 1) % 50 == 0:
            print(f"  ✓ {i + 1}/{len(matched_pairs)} files")

    print(f"\nExtracted {len(all_segments)} segments with timestamps\n")
    return all_segments

## Zone 3: Audio Segmentation

In [32]:
"""
AUDIO SEGMENTER WITH MAX LIMIT
Permet de limiter le nombre de segments à traiter pour respecter les limites de calcul
"""
import subprocess
from pathlib import Path
from typing import List, Dict, Tuple

class AudioSegmenter:
    """Segment audio files based on timestamps with optional limit"""

    def __init__(self, output_dir: Path, sample_rate: int = 16000):
        self.output_dir = output_dir
        self.sample_rate = sample_rate
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.speaker_dirs = {}

    def _get_speaker_dir(self, speaker: str) -> Path:
        """Get or create speaker directory"""
        if speaker not in self.speaker_dirs:
            d = self.output_dir / speaker
            d.mkdir(exist_ok=True)
            self.speaker_dirs[speaker] = d
        return self.speaker_dirs[speaker]

    def extract_segment(self, audio_file: Path, start_ms: int, end_ms: int, output_path: Path) -> bool:
        """Extract audio segment using ffmpeg"""
        if not audio_file.exists():
            return False

        start_sec = start_ms / 1000.0
        duration_sec = (end_ms - start_ms) / 1000.0

        cmd = [
            "ffmpeg", "-i", str(audio_file),
            "-ss", str(start_sec), "-t", str(duration_sec),
            "-acodec", "pcm_s16le", "-ar", str(self.sample_rate), "-ac", "1",
            "-y", str(output_path)
        ]

        try:
            subprocess.run(cmd, check=True, capture_output=True, timeout=10)
            return True
        except:
            return False

    def segment_all(self, segments: List[WorSegment], matched_pairs: List[Tuple[Path, Path]],
                   max_segments: int = None) -> Dict:
        """
        Segment audio files with optional limit

        Args:
            segments: List of WorSegment objects
            matched_pairs: List of (cha_file, audio_file) tuples
            max_segments: Maximum number of segments to process (None = all)

        Returns:
            Dict with:
            - extracted: List of successfully extracted segments
            - skipped: Number of skipped segments (no audio file)
            - stopped_at: Number processed before stopping (if max reached)
            - limited: Boolean, whether max_segments was reached
        """

        print("="*70)
        print(" STEP 3: SEGMENT AUDIO FILES")
        print("="*70)

        audio_lookup = {audio.stem: audio for _, audio in matched_pairs}

        # Déterminer le nombre de segments à traiter
        segments_to_process = segments
        limited = False

        if max_segments is not None:
            if len(segments) > max_segments:
                segments_to_process = segments[:max_segments]
                limited = True
                print(f"\n LIMIT SET: Processing {max_segments}/{len(segments)} segments")
            else:
                print(f"\n Processing all {len(segments)} segments (limit: {max_segments})")
        else:
            print(f"\n Processing all {len(segments)} segments (no limit)")

        results = []
        skipped = 0

        for i, seg in enumerate(segments_to_process):
            audio_file = audio_lookup.get(seg.file_name)
            if not audio_file or not audio_file.exists():
                skipped += 1
                continue

            segment_id = f"{seg.file_name}_{seg.speaker}_{i:05d}"
            speaker_dir = self._get_speaker_dir(seg.speaker)
            output_path = speaker_dir / f"{segment_id}.wav"

            start_ms = seg.words[0][1]
            end_ms = seg.words[-1][2]

            if self.extract_segment(audio_file, start_ms, end_ms, output_path):
                results.append({
                    "segment_id": segment_id,
                    "speaker": seg.speaker,
                    "file_name": seg.file_name,
                    "audio_path": str(output_path),
                    "duration_ms": end_ms - start_ms,
                    "text": seg.text,
                    "num_words": len(seg.words)
                })

            if (i + 1) % 200 == 0:
                print(f"  ✓ {i + 1}/{len(segments_to_process)} segments")

        print(f"\n Segmented {len(results)} audio files")
        if skipped > 0:
            print(f"   Skipped (no audio): {skipped}")

        if limited:
            print(f"   STOPPED AT LIMIT: {len(segments_to_process)} processed")
            print(f"   Remaining: {len(segments) - len(segments_to_process)} segments not processed")

        print()

        return {
            "extracted": results,
            "skipped": skipped,
            "stopped_at": len(segments_to_process),
            "limited": limited,
            "total_segments": len(segments)
        }

# ============================================================================
# HELPER: Resume segmentation
# ============================================================================

def resume_segmentation(segments: List[WorSegment], matched_pairs: List[Tuple[Path, Path]],
                       output_dir: Path, start_from: int = 0, max_segments: int = None) -> Dict:
    """
    Resume segmentation from a specific point

    Utile si tu veux reprendre après avoir atteint ta limite

    Args:
        segments: List of all WorSegment objects
        matched_pairs: List of (cha_file, audio_file) tuples
        output_dir: Output directory
        start_from: Index to start from (0 = beginning)
        max_segments: Maximum segments to process from start_from

    Returns:
        Dict with results
    """

    print("="*70)
    print("RESUME SEGMENTATION")
    print("="*70)
    print(f"\n  Resuming from segment {start_from}")
    print(f"  Total remaining: {len(segments) - start_from}\n")

    # Get remaining segments
    remaining_segments = segments[start_from:]

    segmenter = AudioSegmenter(output_dir)
    results = segmenter.segment_all(remaining_segments, matched_pairs, max_segments)

    # Adjust stopped_at to reflect actual position in original list
    results["stopped_at"] = start_from + results["stopped_at"]

    return results

## Zone 4: Extract Speakers Metadata

In [33]:
def extract_all_speakers_info(matched_pairs: List[Tuple[Path, Path]]) -> Dict[str, str]:
    """Extract speaker roles from all .cha files"""

    all_speakers = {}

    for cha_file, _ in matched_pairs:
        with cha_file.open(encoding="utf-8") as f:
            for line in f:
                if line.startswith("@Participants:"):
                    participants_str = line.split(":", 1)[1].strip()
                    for participant in participants_str.split(","):
                        participant = participant.strip()
                        parts = participant.rsplit(" ", 1)
                        if len(parts) == 2:
                            speaker_name, role = parts
                            all_speakers[speaker_name] = role

    return all_speakers

## Zone 5: Whisper Baseline Evaluation Compute of the WER before fine-tuning

### Whisper Evaluation Class

In [34]:
@dataclass
class TranscriptionResult:
    segment_id: str
    speaker: str
    file_name: str
    audio_path: str
    ground_truth: str
    whisper_prediction: str
    duration_ms: int
    wer: float
    confidence: float = 0.0


class WhisperEvaluator:
    """Evaluate Whisper baseline on children voices"""

    def __init__(self, model_name: str = "base"):
        print(f"Loading Whisper '{model_name}'...")
        self.model = whisper.load_model(model_name)
        print("Loaded\n")

    def transcribe(self, audio_path: Path) -> Dict:
        if not audio_path.exists():
            return {"text": "", "confidence": 0.0}
        try:
            result = self.model.transcribe(str(audio_path), language="fr", verbose=False)
            return {"text": result["text"].strip(), "confidence": result.get("confidence", 0.0)}
        except:
            return {"text": "", "confidence": 0.0}

    @staticmethod
    def calculate_wer(ground_truth: str, prediction: str) -> float:
        if not ground_truth.strip():
            return 0.0 if not prediction.strip() else 1.0
        return compute_wer(ground_truth, prediction)

    def evaluate_children(self, audio_segments: List[Dict], speakers_info: Dict,
                         sample_size: int = None) -> List[TranscriptionResult]:
        """Evaluate only children speakers"""

        print("="*70)
        print("STEP 4: WHISPER BASELINE EVALUATION (CHILDREN ONLY)")
        print("="*70)

        # Filter children only
        children_segments = [s for s in audio_segments
                            if speakers_info.get(s["speaker"]) in CHILD_ROLES]

        if sample_size:
            children_segments = children_segments[:sample_size]

        print(f"\nEvaluating {len(children_segments)} children segments\n")

        results = []
        for i, seg in enumerate(children_segments):
            audio_path = Path(seg["audio_path"])
            transcription = self.transcribe(audio_path)
            wer = self.calculate_wer(seg["text"], transcription["text"])

            results.append(TranscriptionResult(
                segment_id=seg["segment_id"],
                speaker=seg["speaker"],
                file_name=seg["file_name"],
                audio_path=seg["audio_path"],
                ground_truth=seg["text"],
                whisper_prediction=transcription["text"],
                duration_ms=seg["duration_ms"],
                wer=wer,
                confidence=transcription["confidence"]
            ))

            if (i + 1) % 50 == 0:
                avg_wer = sum(r.wer for r in results) / len(results)
                print(f"  ✓ {i + 1}/{len(children_segments)} | Avg WER: {avg_wer:.3f}")

        return results


### WER Report

In [35]:

def print_wer_report(results: List[TranscriptionResult], speakers_info: Dict):
    """Print WER statistics"""

    wers = [r.wer for r in results]

    print("\n" + "="*70)
    print("STEP 5: WER STATISTICS (CHILDREN ONLY)")
    print("="*70)

    print(f"\nGlobal:")
    print(f"   Total segments:  {len(results)}")
    print(f"   Avg WER:         {sum(wers) / len(wers):.3f}")
    print(f"   Min WER:         {min(wers):.3f}")
    print(f"   Max WER:         {max(wers):.3f}")
    print(f"   Median WER:      {sorted(wers)[len(wers)//2]:.3f}")

    print(f"\n   Distribution:")
    for low, high in [(0.0, 0.1), (0.1, 0.3), (0.3, 0.5), (0.5, 1.0)]:
        count = sum(1 for w in wers if low <= w < high)
        pct = (count / len(wers)) * 100
        print(f"      {low:.1f}-{high:.1f}: {count:4d} ({pct:5.1f}%)")

    # By speaker
    by_speaker = {}
    for r in results:
        by_speaker.setdefault(r.speaker, []).append(r.wer)

    print(f"\n By speaker ({len(by_speaker)}):")
    for speaker in sorted(by_speaker.keys()):
        wers_sp = by_speaker[speaker]
        avg = sum(wers_sp) / len(wers_sp)
        print(f"      {speaker:15} {len(wers_sp):4d} segments | WER: {avg:.3f}")

    print("\n" + "="*70 + "\n")


## Zone 6: Create Training Dataset

In [36]:
class DatasetBuilder:
    """Create train/test splits for fine-tuning"""

    def __init__(self, output_dir: Path):
        self.output_dir = output_dir
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def create_dataset(self, results: List[TranscriptionResult], train_ratio: float = 0.8):
        """Create JSONL + metadata files"""

        print("="*70)
        print("STEP 6: CREATE TRAINING DATASET")
        print("="*70)

        split_idx = int(len(results) * train_ratio)
        train = results[:split_idx]
        test = results[split_idx:]

        print(f"\nDataset split:")
        print(f"   Total:   {len(results)} segments")
        print(f"   Train:   {len(train)} segments ({train_ratio*100:.0f}%)")
        print(f"   Test:    {len(test)} segments ({(1-train_ratio)*100:.0f}%)")

        # Save JSONL (for fine-tuning)
        self._save_jsonl(train, self.output_dir / "train.jsonl")
        self._save_jsonl(test, self.output_dir / "eval.jsonl")

        # Save metadata JSON (for analysis)
        self._save_metadata(train, self.output_dir / "train_metadata.json")
        self._save_metadata(test, self.output_dir / "eval_metadata.json")

        print(f"\nDataset created in {self.output_dir}\n")

    def _save_jsonl(self, results: List[TranscriptionResult], output_file: Path):
        """Save as JSONL for Whisper"""
        with open(output_file, "w", encoding="utf-8") as f:
            for r in results:
                entry = {"audio": r.audio_path, "text": r.ground_truth, "language": "fr"}
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")
        print(f"   ✓ {output_file.name} ({len(results)} segments)")

    def _save_metadata(self, results: List[TranscriptionResult], output_file: Path):
        """Save complete metadata"""
        data = [asdict(r) for r in results]
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        print(f"   ✓ {output_file.name}")

## MAIN PIPELINE

In [37]:
def run_pipeline():
    """Run complete pipeline end-to-end"""

    print("\n" + "="*70)
    print("COMPLETE WHISPER CHILDREN DATASET PIPELINE")
    print("="*70)

    # ZONE 1: Matching
    match_result = find_matching_files(CONFIG["cha_dir"], CONFIG["audio_dir"], CONFIG["audio_extensions"])
    print_matching_report(match_result)

    if not match_result["matched"]:
        print(" No matched pairs found!")
        return

    # ZONE 2: Extract segments
    segments = extract_segments_from_matched(match_result["matched"])

    # ZONE 3: Segment audio
    segmenter = AudioSegmenter(CONFIG["output_dir"] / "audio_segments")
    segment_all_result = segmenter.segment_all(segments, match_result["matched"], max_segments=100)
    audio_segments = segment_all_result["extracted"]

    # ZONE 4: Get speakers info
    speakers_info = extract_all_speakers_info(match_result["matched"])

    # ZONE 5: Evaluate Whisper (children only)
    evaluator = WhisperEvaluator(CONFIG["whisper_model"])
    results = evaluator.evaluate_children(audio_segments, speakers_info, CONFIG["sample_size"])

    print_wer_report(results, speakers_info)

    # ZONE 6: Create dataset
    builder = DatasetBuilder(CONFIG["output_dir"] / "training_dataset")
    builder.create_dataset(results, CONFIG["train_ratio"])

    print("="*70)
    print("PIPELINE COMPLETE")
    print("="*70)
    print(f"\nOutput directory: {CONFIG['output_dir']}")
    print("\nFiles generated:")
    print("   training_dataset/train.jsonl ........... for fine-tuning")
    print("   training_dataset/eval.jsonl ........... for evaluation")
    print("   training_dataset/train_metadata.json .. complete metadata")
    print("   training_dataset/eval_metadata.json ... complete metadata")
    print("\nReady for Whisper fine-tuning!\n")


if __name__ == "__main__":
    run_pipeline()


COMPLETE WHISPER CHILDREN DATASET PIPELINE

STEP 1: FILE MATCHING (by relative path)

 Found:
   Total .cha files:       112
   Total audio files:      112
   Matched pairs:        112
   .cha missing audio:  0
   Audio orphans:       0


STEP 2: EXTRACT .CHA SEGMENTS
  ✓ 50/112 files
  ✓ 100/112 files

Extracted 13992 segments with timestamps

🎵 STEP 3: SEGMENT AUDIO FILES

⚠️  LIMIT SET: Processing 100/13992 segments

✅ Segmented 100 audio files
   ⚠️  STOPPED AT LIMIT: 100 processed
   Remaining: 13892 segments not processed

Loading Whisper 'base'...
Loaded

STEP 4: WHISPER BASELINE EVALUATION (CHILDREN ONLY)

Evaluating 29 children segments



  0%|          | 0/372 [00:00<?, ?frames/s]


STEP 5: WER STATISTICS (CHILDREN ONLY)

Global:
   Total segments:  29
   Avg WER:         1.314
   Min WER:         0.800
   Max WER:         4.889
   Median WER:      1.000

   Distribution:
      0.0-0.1:    0 (  0.0%)
      0.1-0.3:    0 (  0.0%)
      0.3-0.5:    0 (  0.0%)
      0.5-1.0:    2 (  6.9%)

 By speaker (6):
      CHI                2 segments | WER: 1.786
      ELI                4 segments | WER: 2.056
      MAT               17 segments | WER: 1.135
      RIT                2 segments | WER: 1.500
      UNI                3 segments | WER: 1.000
      WIL                1 segments | WER: 1.000


STEP 6: CREATE TRAINING DATASET

Dataset split:
   Total:   29 segments
   Train:   23 segments (80%)
   Test:    6 segments (20%)
   ✓ train.jsonl (23 segments)
   ✓ eval.jsonl (6 segments)
   ✓ train_metadata.json
   ✓ eval_metadata.json

Dataset created in /content/drive/MyDrive/asr/output/whisper_children_dataset/training_dataset

PIPELINE COMPLETE

Output directory: /c